## Neural Collaborative Filtering 
* 저자 code : https://github.com/hexiangnan/neural_collaborative_filtering
* Keras docs reference : https://keras.io/examples/structured_data/collaborative_filtering_movielens/
* paper 에서는 implicit feedback 을 학습하는 것을 실험했으나, 아래 실습에서는 직접 rating prediction 을 진행